In [39]:
!head -n 5 datasets/transfusion.data

'head' is not recognized as an internal or external command,
operable program or batch file.


In [40]:
import pandas as pd
transfusion=pd.read_csv("transfusion.data")
transfusion.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [41]:
transfusion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 5 columns):
 #   Column                                      Non-Null Count  Dtype
---  ------                                      --------------  -----
 0   Recency (months)                            748 non-null    int64
 1   Frequency (times)                           748 non-null    int64
 2   Monetary (c.c. blood)                       748 non-null    int64
 3   Time (months)                               748 non-null    int64
 4   whether he/she donated blood in March 2007  748 non-null    int64
dtypes: int64(5)
memory usage: 29.3 KB


In [42]:
transfusion.rename(columns={'whether he/she donated blood in March 2007':'target'},inplace=True)
transfusion.head(2)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),target
0,2,50,12500,98,1
1,0,13,3250,28,1


In [43]:
transfusion.target.value_counts(normalize=True).round(3)

target
0    0.762
1    0.238
Name: proportion, dtype: float64

In [44]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(
    transfusion.drop(columns='target'),
    transfusion.target,
    test_size=0.25,
    random_state=42,
    stratify=transfusion.target
)
x_train.head(2)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
334,16,2,500,16
99,5,7,1750,26


In [45]:
from tpot import TPOTClassifier
from sklearn.metrics import roc_auc_score
tpot=TPOTClassifier(
    generations=5,
    population_size=20,
    verbosity=2,
    scoring='roc_auc',
    random_state=42,
    disable_update_check=True,
    config_dict='TPOT light'
)
tpot.fit(x_train,y_train)
tpot_auc_score=roc_auc_score(y_test,tpot.predict_proba(x_test)[:,1])
print(f'\nAUC score:{tpot_auc_score:.4f}')
print('\nBest pipeline steps:',end='\n')
for idx,(name,transform) in enumerate(tpot.fitted_pipeline_.steps,start=1):
    print(f'{idx}.{transform}')

Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7422459184429089

Generation 2 - Current best internal CV score: 0.7422459184429089

Generation 3 - Current best internal CV score: 0.7423330644124078

Generation 4 - Current best internal CV score: 0.7423330644124078

Generation 5 - Current best internal CV score: 0.7423330644124078

Best pipeline: LogisticRegression(RobustScaler(input_matrix), C=25.0, dual=False, penalty=l2)

AUC score:0.7858

Best pipeline steps:
1.RobustScaler()
2.LogisticRegression(C=25.0, random_state=42)


In [46]:
x_train.var().round(3)

Recency (months)              66.929
Frequency (times)             33.830
Monetary (c.c. blood)    2114363.700
Time (months)                611.147
dtype: float64

In [47]:
import numpy as np
x_train_normed,x_test_normed=x_train.copy(),x_test.copy()
col_to_normalize= 'Monetary (c.c. blood)'
for df_ in [x_train_normed,x_test_normed]:
    df_['monetary_log']=np.log(df_[col_to_normalize])
    df_.drop(columns=col_to_normalize,inplace=True)
x_train_normed.var()

Recency (months)      66.929017
Frequency (times)     33.829819
Time (months)        611.146588
monetary_log           0.837458
dtype: float64

In [48]:
from sklearn import linear_model
logreg=linear_model.LogisticRegression(
    solver='liblinear',
    random_state=42
)
logreg.fit(x_train_normed,y_train)
logreg_auc_score=roc_auc_score(y_test,logreg.predict_proba(x_test_normed)[:,1])
print(f'\nAUC score:{logreg_auc_score:.4f}')


AUC score:0.7891


In [49]:
# Importing itemgetter
from operator import itemgetter

# Sort models based on their AUC score from highest to lowest
sorted(
    [('tpot', tpot_auc_score), ('logreg', logreg_auc_score)],
    key=itemgetter(1),
    reverse=True)

[('logreg', 0.7890972663699937), ('tpot', 0.7857596948506039)]